##Data comes from   
https://www.kaggle.com/austinreese/craigslist-carstrucks-data

In [0]:
import pandas as pd
import numpy as np
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Go to link, sign in, and paste in result

In [0]:
file_list = drive.ListFile({'q': "'1kE03N5LwmWDDsDescCPmlQxGsY7ZKEnT' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: craigslist_cleaned_df_img.csv, id: 1q_L-LMSE29jp64wP32VvJ8brX23L88NM
title: craigslist_df_cleaned2.csv, id: 1ZdS5bfNwTJVYVsFoPTBRDY5BoBmh6Ctp
title: craigslist_df_cleaned, id: 1NT6IPKm1EwNeH5PBVt8HqRMtO_oJemnC
title: Weekly_U.S._All_Grades_All_Formulations_Retail_Gasoline_Prices, id: 1cmGfdjaAd7RXYa_9S8RcraEDBA1fJKoKzB17KcaEo7Y
title: Weekly_U.S._All_Grades_All_Formulations_Retail_Gasoline_Prices.csv, id: 1pBHxMMMbUXZfFhNj5c2KqFq_dm4EkYOK
title: vehicles.csv, id: 12aa_5F_dECaFGjQR5KYZwToAqhHouUNi
title: carfullspecs.csv, id: 1Y9oQ0huvw3LYnbafa-AM7Ig5cOK95DkD
title: vehicles_cl.csv, id: 1BKZfd5WBLkryOSWI6t4g50bloTD8P6f3


In [0]:
downloaded = drive.CreateFile({'id':"1BKZfd5WBLkryOSWI6t4g50bloTD8P6f3"})  
downloaded.GetContentFile('vehicles_cl.csv')       

In [0]:
df_orig = pd.read_csv('vehicles_cl.csv')
df_orig

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7088746062,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,10299,2012.0,acura,tl,NaN,NaN,gas,90186.0,clean,automatic,19UUA8F22CA003926,NaN,NaN,other,blue,https://images.craigslist.org/01414_3LIXs9EO33...,2012 Acura TL Base 4dr Sedan Offered by: B...,NaN,nc,35.7636,-78.7443
1,7088745301,https://greensboro.craigslist.org/ctd/d/bmw-3-...,greensboro,https://greensboro.craigslist.org,0,2011.0,bmw,335,NaN,6 cylinders,gas,115120.0,clean,automatic,NaN,rwd,NaN,convertible,blue,https://images.craigslist.org/00S0S_1kTatLGLxB...,BMW 3 Series 335i Convertible Navigation Dakot...,NaN,nc,NaN,NaN
2,7088744126,https://greensboro.craigslist.org/cto/d/greens...,greensboro,https://greensboro.craigslist.org,9500,2011.0,jaguar,xf,excellent,NaN,gas,85000.0,clean,automatic,NaN,NaN,NaN,NaN,blue,https://images.craigslist.org/00505_f22HGItCRp...,2011 jaguar XF premium - estate sale. Retired ...,NaN,nc,36.1032,-79.8794
3,7088743681,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,3995,2004.0,honda,element,NaN,NaN,gas,212526.0,clean,automatic,5J6YH18314L006498,fwd,NaN,SUV,orange,https://images.craigslist.org/00E0E_eAUnhFF86M...,2004 Honda Element LX 4dr SUV Offered by: ...,NaN,nc,35.7636,-78.7443
4,7074612539,https://lincoln.craigslist.org/ctd/d/gretna-20...,lincoln,https://lincoln.craigslist.org,41988,2016.0,chevrolet,silverado k2500hd,NaN,NaN,gas,NaN,clean,automatic,1GC1KWE85GF266427,NaN,NaN,NaN,NaN,https://images.craigslist.org/00S0S_8msT7RQquO...,"Shop Indoors, Heated Showroom!!!www.gretnaauto...",NaN,ne,41.1345,-96.2458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539754,7076296329,https://syracuse.craigslist.org/cto/d/mottvill...,syracuse,https://syracuse.craigslist.org,7950,2008.0,rover,NaN,excellent,8 cylinders,gas,NaN,clean,automatic,NaN,4wd,NaN,NaN,NaN,https://images.craigslist.org/00T0T_86RgcbSIhT...,Selling my 2008 Range Rover - it's an extra ve...,NaN,ny,42.9506,-76.4254
539755,7088752165,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,13995,2016.0,nissan,frontier,NaN,NaN,gas,97545.0,clean,automatic,1N6AD0CW7GN748778,4wd,NaN,pickup,silver,https://images.craigslist.org/00R0R_2FiHQj8mDm...,2016 Nissan Frontier SV V6 4x4 4dr King Cab 6....,NaN,nc,35.7636,-78.7443
539756,7088749883,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,10995,2010.0,bmw,335i,NaN,NaN,gas,80328.0,clean,automatic,WBAWB7C58AP049201,NaN,compact,coupe,black,https://images.craigslist.org/00v0v_3sp1G5hNhk...,2010 BMW 3 Series 335i 2dr Coupe Offered b...,NaN,nc,35.7636,-78.7443
539757,7088747986,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,6495,2015.0,dodge,dart,NaN,NaN,other,83521.0,clean,manual,1C3CDFAA5FD252294,fwd,NaN,sedan,red,https://images.craigslist.org/00101_6orUUSDpdh...,2015 Dodge Dart SE 4dr Sedan Offered by: B...,NaN,nc,35.7636,-78.7443


In [0]:
df_orig.isnull().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year               987
manufacturer     23584
model             8013
condition       236052
cylinders       218495
fuel              3393
odometer         98976
title_status      2940
transmission      3973
vin             224410
drive           155772
size            371209
type            147469
paint_color     174239
image_url           19
description         21
county          539759
state                0
lat               8974
long              8974
dtype: int64

In [0]:
df_img = df_orig.copy()
df_img = df_img[['price', 'year', 'manufacturer', 'model', 'image_url', 'odometer', 'state', 'condition']]
df_img.head()

,price,year,manufacturer,model,image_url,odometer,state,condition
0,10299,2012.0,acura,tl,https://images.craigslist.org/01414_3LIXs9EO33...,90186.0,nc,NaN
1,0,2011.0,bmw,335,https://images.craigslist.org/00S0S_1kTatLGLxB...,115120.0,nc,NaN
2,9500,2011.0,jaguar,xf,https://images.craigslist.org/00505_f22HGItCRp...,85000.0,nc,excellent
3,3995,2004.0,honda,element,https://images.craigslist.org/00E0E_eAUnhFF86M...,212526.0,nc,NaN
4,41988,2016.0,chevrolet,silverado k2500hd,https://images.craigslist.org/00S0S_8msT7RQquO...,NaN,ne,NaN


In [0]:
import matplotlib.pyplot as plt

df_img[df_img['price'] >= np.percentile(df_img['price'], 99)].sort_values('price', ascending=False).head(150)

,price,year,manufacturer,model,image_url,odometer,state,condition
174324,189899.0,2009.0,mercedes-benz,sl-class,https://images.craigslist.org/00K0K_5ckEuT3WJa...,7474.0,fl,NaN
347213,189000.0,2014.0,ford,f-150,https://images.craigslist.org/00F0F_2SPRjdbjvc...,151248.0,al,good
316996,189000.0,2014.0,ford,f-150,https://images.craigslist.org/00F0F_2SPRjdbjvc...,151248.0,al,good
14370,189000.0,2014.0,ford,f-150,https://images.craigslist.org/00F0F_2SPRjdbjvc...,151248.0,ms,good
17897,189000.0,2014.0,ford,f-150,https://images.craigslist.org/00F0F_2SPRjdbjvc...,151248.0,ms,good
...,...,...,...,...,...,...,...,...
214236,99500.0,2019.0,ford,f-550,https://images.craigslist.org/00L0L_dU5225uwOA...,19835.0,nd,NaN
32087,99500.0,2019.0,ford,f-550,https://images.craigslist.org/00L0L_dU5225uwOA...,19835.0,ny,NaN
451633,99500.0,2019.0,ford,f-550,https://images.craigslist.org/00L0L_dU5225uwOA...,19835.0,pa,NaN
295922,99500.0,2019.0,ford,f-550,https://images.craigslist.org/00L0L_dU5225uwOA...,19835.0,ga,NaN


In [0]:
df_img['image_url'][428730]

'https://images.craigslist.org/00n0n_70hfkXLhvkB_600x450.jpg'

In [0]:
## Cleaning

df_img = df_img[(df_img['price'] >= 500) &
          (df_img['price'] < 200000)]
df_img = df_img.dropna(subset=['price', 'manufacturer', 'model', 'year'])
df_img = df_img.groupby("model").filter(lambda x: len(x) > 2)
df_img = df_img[df_img.year >= 2000]
df_img.shape

(413503, 8)

In [0]:
df_img['manufacturer'] = df_img['manufacturer'].str.lower()
df_img['model'] = df_img['model'].str.lower()
df_img['state'] = df_img['state'].str.lower()

In [0]:
df_img.to_csv('df_std_clean.csv')

In [0]:
df_img_slim = pd.read_csv("df_std_clean.csv")

In [0]:
df_img_slim = df_img_slim[['model', 'image_url']]

In [0]:
df_img_slim.to_csv("model_and_image_url_lookup.csv")